In [10]:
from elasticsearch import Elasticsearch

# 連接 Elasticsearch（無需帳號密碼）
es = Elasticsearch("http://localhost:9200")

# 測試連線
if es.ping():
    print("✅ 成功連接 Elasticsearch")
else:
    print("❌ 連接失敗")


✅ 成功連接 Elasticsearch


In [18]:
index_name = "text_experiment"

# 如果索引已經存在，則刪除並重新建立
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "content": {"type": "text"},
            "created_at": {"type": "date"}
        }
    }
})

print(f"✅ 索引 '{index_name}' 已建立")


✅ 索引 'text_experiment' 已建立


In [21]:
import datetime

# 要插入的文本資料
doc = {
    "title": "義大利麵店",
    "content": "好吃的義大利麵店",
    "created_at": datetime.datetime.now()
}

# 插入文件（Elasticsearch 會自動生成 ID）
res = es.index(index=index_name, body=doc)
print(f"✅ 文本已寫入，ID: {res['_id']}")


✅ 文本已寫入，ID: XWLZ1ZQBGJlcZ1djSwVL


In [23]:
query = {
    "query": {
        "match": {
            "content": "義大利"
        }
    }
}

response = es.search(index=index_name, body=query)

print("🔍 搜索結果：")
for hit in response["hits"]["hits"]:
    print(f"📄 {hit['_source']['title']} (ID: {hit['_id']})")


🔍 搜索結果：
📄 義大利麵店 (ID: XWLZ1ZQBGJlcZ1djSwVL)


In [16]:
# 先搜尋最新的一篇文章
query = {"query": {"match_all": {}}}
response = es.search(index=index_name, body=query)

if response["hits"]["hits"]:
    doc_id = response["hits"]["hits"][0]["_id"]  # 取得 ID
    es.delete(index=index_name, id=doc_id)
    print(f"🗑️ 文本已刪除，ID: {doc_id}")
else:
    print("❌ 沒有可刪除的文本")


🗑️ 文本已刪除，ID: WGLU1ZQBGJlcZ1dj8AVv


In [17]:
es.indices.delete(index=index_name)
print(f"🗑️ 索引 '{index_name}' 已刪除")


🗑️ 索引 'text_experiment' 已刪除
